In [1]:
import os
import pandas as pd

# Functions to compute the numerical results

In [2]:
def load_metrics(result_folder, split):
    if "LFLR" in result_folder:
        metrics_file = os.path.join(result_folder, "LR", f"split_{split}", f"metrics_split{split}.csv")
    elif "LFCO" in result_folder:
        metrics_file = os.path.join(result_folder, "LR", f"split_{split}", f"metrics_split{split}.csv")
    else:
        metrics_file = os.path.join(result_folder, f"split_{split}", f"metrics_split{split}.csv")
    
    if not os.path.exists(metrics_file):
        raise FileNotFoundError(f"File not found: {metrics_file}")
    
    return pd.read_csv(metrics_file)

# Function to calculate the mean and standard deviation of metrics
def calculate_statistics(result_folder, splits=[1, 2, 3]):
    all_metrics = []

    # Load metrics from all splits
    for split in splits:
        split_metrics = load_metrics(result_folder, split)
        all_metrics.append(split_metrics)

    # Concatenate all split metrics into a DataFrame
    metrics_df = pd.concat(all_metrics)

    # Calculate mean and standard deviation for each metric
    statistics = {
        "mean": metrics_df.mean() * 100,
        "std_dev": metrics_df.std() * 100
    }

    return statistics

def get_valid_subfolders(directory):
    return sorted([f for f in os.listdir(directory) if os.path.isdir(os.path.join(directory, f)) and not f.startswith('.')])

# Results

In [3]:

models = ["./MLP", "./JHF", "./GRU", './LFCO', './FHSI', './LFLR', './Transformer']

results_list = []

for model in models:
    result_subfolders = get_valid_subfolders(model)
    
    for result_folder in result_subfolders:
        result_path = os.path.join(model, result_folder)

        try:
            statistics = calculate_statistics(result_path)
            mean = statistics["mean"]
            std_dev = statistics["std_dev"]
            row = {
                "Model": f"{os.path.basename(model)}_{result_folder}",
                "Accuracy": f"{mean['accuracy']:.2f} ± {std_dev['accuracy']:.2f}",
                "Specificity": f"{mean['specificity']:.2f} ± {std_dev['specificity']:.2f}",
                "Sensitivity": f"{mean['recall']:.2f} ± {std_dev['recall']:.2f}",
                "ROC AUC": f"{mean['roc_auc']:.2f} ± {std_dev['roc_auc']:.2f}"
            }
            
            results_list.append(row)
        
        except FileNotFoundError as e:
            print(e)

results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values(by="Model").reset_index(drop=True)

In [4]:
results_df

,Model,Accuracy,Specificity,Sensitivity,ROC AUC
0,FHSI_Results_FHSI-2MTS-3Features,70.31 ± 1.29,69.52 ± 1.84,73.35 ± 0.64,75.05 ± 1.67
1,FHSI_Results_FHSI-3MTS-3Features,69.41 ± 0.60,68.61 ± 0.61,72.54 ± 0.99,74.67 ± 0.96
2,FHSI_Results_FHSI-4MTS-3Features,71.47 ± 0.46,70.59 ± 0.20,74.96 ± 1.71,76.49 ± 2.25
3,FHSI_Results_FHSI-5MTS-3Features,69.57 ± 3.39,69.01 ± 4.95,71.89 ± 3.31,76.90 ± 3.10
4,GRU_Results_GRU-2MTS,72.42 ± 0.18,74.17 ± 1.27,65.69 ± 4.26,73.69 ± 0.39
5,GRU_Results_GRU-3MTS,71.68 ± 2.36,73.00 ± 3.82,66.28 ± 4.64,73.42 ± 1.40
6,GRU_Results_GRU-4MTS,67.46 ± 4.31,66.96 ± 5.68,69.17 ± 3.21,71.86 ± 1.85
7,GRU_Results_GRU-5MTS,69.25 ± 1.35,69.86 ± 0.59,67.03 ± 4.62,72.47 ± 1.56
8,JHF_Results_JHF-3MTS-3Features,69.04 ± 2.72,68.85 ± 5.11,70.29 ± 7.68,75.14 ± 1.47
9,JHF_Results_JHF-4MTS-3Features,69.67 ± 3.21,70.25 ± 5.27,67.85 ± 5.45,74.30 ± 0.65


In [5]:
import pandas as pd
gcnn_3 = pd.read_csv('./GNNs/Results_3MTS/results.csv')
multgcnn_3 = pd.read_csv('./MultGNN/Results_3MTS/multimodal_results.csv')

gcnn_4 = pd.read_csv('./GNNs/Results_4MTS/results.csv')
multgcnn_4 = pd.read_csv('./MultGNN/Results_4MTS/multimodal_results.csv')

gcnn_5 = pd.read_csv('./GNNs/Results_5MTS/results.csv')
multgcnn_5 = pd.read_csv('./MultGNN/Results_5MTS/multimodal_results.csv')

gcnn_3['Experiment'] = 'GCNN-3'
multgcnn_3['Experiment'] = 'MultGCNN-3'

gcnn_4['Experiment'] = 'GCNN-4'
multgcnn_4['Experiment'] = 'MultGCNN-4'

gcnn_5['Experiment'] = 'GCNN-5'
multgcnn_5['Experiment'] = 'MultGCNN-5'


In [6]:
import numpy as np

# Last values from each row
last_values = [
    63.07, 62.34, 62.16,  # MLP
    73.42, 71.86, 72.47,  # GRU
    68.51, 68.66, 69.46,  # Transformer
    67.95, 67.89, 66.22,  # GCNN
    75.14, 74.30, 73.69,  # JHF
    74.67, 76.49, 76.90   # FHSI
]

mean_last_values = np.mean(last_values)
std_last_values = np.std(last_values)

# Print the results
print(f"Mean of the last values: {mean_last_values:.2f}")
print(f"Standard Deviation of the last values: {std_last_values:.2f}")


Mean of the last values: 70.29
Standard Deviation of the last values: 4.63
